# Solving Continuous Algebraic Riccati Equations with Python

We want to solve the equation $$A^T P + PA - PBR^{-1}B^TP + Q - \rho P = 0 \tag{$\star$}$$ for the matrix $P$, where $$ \rho = 0.1, \quad A = \begin{bmatrix} 0 & 1 \\ -2 & -3\end{bmatrix}, \quad B = \begin{bmatrix} 0 \\ 1\end{bmatrix}, \quad G = \begin{bmatrix} 0.1 & 0 \\ 0 & 0.1\end{bmatrix}, \quad Q = \begin{bmatrix} 1 & 0 \\ 0 & 1\end{bmatrix}, \quad R = [1]$$

Import the following Python modules:

In [1]:
import numpy as np
from scipy.linalg import solve_continuous_are

Initialize the parameters:

In [8]:
rho = 0.1
A = np.array([[0, 1], [-2, -3]])
B = np.array([[0, 1]]).T
G = 0.1*np.eye(2) 
Q = np.eye(2)
R = np.eye(1)

The function `solve_continuous_are(a, b, q, r, e = None, s = None, balanced = True)` solves the continuous-time algebraic Riccati equation $$A^TX + XA - XBR^{-1}B^T X + Q = 0 \tag{$\star \star$}$$ 

Here, $H$ means the conjugate transpose (which in the case of real matrices, is just transpose). 

Since the equation $(\star \star)$ we can solve with `solve_continuous_are` is in a different form than $(\star)$, we have to modify $A \to A - \frac{\rho}{2} A$ in $(\star \star)$.

In [9]:
A_mod = A - (rho/2)*A

Now, we can call the function.

In [12]:
P = solve_continuous_are(A, B, Q, R)
print(P)

[[1.23606798 0.23606798]
 [0.23606798 0.23606798]]
